In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,New Norfolk,-42.7826,147.0587,42.30,83,13,1.01,AU,2022-07-28 22:14:16
1,1,Hermanus,-34.4187,19.2345,57.33,77,22,12.17,ZA,2022-07-28 22:09:17
2,2,Busselton,-33.6500,115.3333,58.51,71,100,15.70,AU,2022-07-28 22:14:16
3,3,Chifeng,42.2683,118.9636,65.80,81,100,1.70,CN,2022-07-28 22:14:17
4,4,Moron,-34.6534,-58.6198,55.15,67,4,10.47,AR,2022-07-28 22:14:17


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Butaritari,3.0707,172.7902,81.88,72,37,15.97,KI,2022-07-28 22:14:18
10,10,Bandarbeyla,9.4942,50.8122,76.17,82,74,26.80,SO,2022-07-28 22:14:19
13,13,Bud,39.4470,-86.1758,85.98,61,75,4.45,US,2022-07-28 22:14:20
16,16,Bethel,41.3712,-73.4140,86.36,75,100,6.91,US,2022-07-28 22:14:21
20,20,Dingle,10.9995,122.6711,77.81,90,100,7.20,PH,2022-07-28 22:14:22
23,23,Padang,-0.9492,100.3543,78.01,85,100,4.59,ID,2022-07-28 22:14:23
24,24,Takoradi,4.8845,-1.7554,75.38,79,78,10.96,GH,2022-07-28 22:14:23
26,26,Tateyama,34.9833,139.8667,83.10,85,84,8.86,JP,2022-07-28 22:14:24
27,27,Eyl,7.9803,49.8164,76.15,78,66,21.12,SO,2022-07-28 22:14:24
28,28,Avarua,-21.2078,-159.7750,78.85,78,20,21.85,CK,2022-07-28 22:14:24


In [10]:
preferred_cities_df.count()

City_ID       202
City          202
Lat           202
Lng           202
Max Temp      202
Humidity      202
Cloudiness    202
Wind Speed    202
Country       202
Date          202
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Butaritari,KI,81.88,3.0707,172.7902,
10,Bandarbeyla,SO,76.17,9.4942,50.8122,
13,Bud,US,85.98,39.4470,-86.1758,
16,Bethel,US,86.36,41.3712,-73.4140,
20,Dingle,PH,77.81,10.9995,122.6711,
23,Padang,ID,78.01,-0.9492,100.3543,
24,Takoradi,GH,75.38,4.8845,-1.7554,
26,Tateyama,JP,83.10,34.9833,139.8667,
27,Eyl,SO,76.15,7.9803,49.8164,
28,Avarua,CK,78.85,-21.2078,-159.7750,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [14]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'AeJbb3dxBygwGngwt2q62zlP3wUoIGg4dM9F1iX4UJzVvNyfi8-jXFWNzQE9wjmj3UEZRRuo9zmdo-VIxDdXtOSPSmN55kAEYSQgz1YAbYn7xoFRdGfu28eZPHnEwHUkD0iKPRUriDgLonaQ1jl2aBnFMRGRPDAzOhwfw5CLW4tNTSlrVeRyjt-UkLGRsOKTf63BXC-0TD0OvodwwNJ_224iUnljIMlqkQDQ7ASOQz8yr0gmd87bOkVcg_F0fHeMAnaeDnmyp8OBC8rkE0AOyiEvW7NIi3Gr-cAUXaRFSesKzrIkxizuXF1bYSQ0Py9_5O7N7CWRVZw7LNXu2zHLKBLu9DAyYVutAk3-w9RXguvxxvkWkV0ey5oz11EPLsNt6fvMOxOYBCv8_Ok-fRaOESG6EoCzYLgeubsr0IMJr1-AlMVeEgvP6sbVYlM',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'name

In [15]:
len(hotels["results"])

20

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))